In [1]:
import json
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import functools

In [27]:
dataset_name = "minds"

In [28]:
if dataset_name == "minds":
#     refs = [str(i) for i in range(39, 44)]
    refs = [str(i) for i in range(49, 54)]
    person_range = range(1, 13)

if dataset_name == "ufop":
    refs = [str(i) for i in range(44, 49)]
    person_range = range(1, 6)

In [29]:
refs

['49', '50', '51', '52', '53']

In [30]:
all_results = {}
for ref in refs:
    results = []
    results_path = f"../99_model_output/results/{ref}/{dataset_name}/"
    for file in os.listdir(results_path):
        if file.endswith(".json"):
            with open(os.path.join(results_path, file), "r") as f:
                results.append(json.load(f))
    all_results[ref] = results

In [31]:
def get_result_by_validation_person(person, results):
    for result in results:
        if result["validate_people"] == [person]:
            return result
    print(f"Not found for person {person}")
    return None

In [32]:
def get_metric(y_true, y_pred, metric, average):
    if metric == accuracy_score:
        return metric(y_true, y_pred)
    else:
        return metric(y_true, y_pred, average=average, zero_division=0)

def get_average_metrics(metric, average):
    values = []
    for r in results:
        y_true = r["true_labels"]
        y_pred = r["predicted_labels"]
        values.append(get_metric(y_true, y_pred, metric, average))
    return values

def print_metrics(metric, average):
    metric_results = get_average_metrics(metric, average)
    print("=" * 10)
    if average is not None:
        print(average, metric.__name__)
    else:
        print(metric.__name__)
    average = sum(metric_results) / len(metric_results)
    std = np.std(metric_results)
    print("Average:", average)
    print("std: ", std)
    print(round(average, 2), "$\pm$", round(std, 2))
    print("=" * 10)

In [33]:
data = []
for validate_person in person_range:
#     print(f"fold {validate_person}")
    accuracies = []
    for ref in refs:
        result = get_result_by_validation_person(validate_person, all_results[ref])
        accuracies.append(max(result["history"]["val_accuracy"]))
    best_ref_index = accuracies.index(max(accuracies))
    best_ref = refs[best_ref_index]
    results = [get_result_by_validation_person(validate_person, all_results[best_ref])]
    
    metric_results = get_average_metrics(accuracy_score, None)
    
    data.append({
        "fold": validate_person,
        "frame": results[0]["meta"]["frames"],
        "val_accuracy": max(accuracies),
        "test_accuracy": metric_results[0]
    })
# data

In [34]:
df = pd.DataFrame.from_dict(data)
df = df.set_index("fold")
df

,frame,val_accuracy,test_accuracy
fold,,,
1,55,0.930000,0.920000
2,69,0.980000,0.929412
3,69,0.929412,0.852632
4,62,0.936842,0.850000
5,55,0.900000,0.990000
6,62,1.000000,0.883495
7,55,0.941748,0.950000
8,69,1.000000,1.000000
9,55,1.000000,0.900000


In [35]:
df["test_accuracy"].mean()

0.9279615407736933

In [36]:
df["frame"].value_counts()

frame
55    6
69    3
62    3
Name: count, dtype: int64